In [ ]:
'''
Investor Network Analysis
'''

In [11]:
import pandas as pd
import re
import gensim
import pickle
from datetime import datetime
from matplotlib import pyplot as plt
import networkx as nx
import numpy as np
import operator
%matplotlib inline

In [12]:
# load data
data_folder = '../data/csv_export/'
util_folder = '../util/'
df_funding_rounds = pd.read_csv(data_folder + 'funding_rounds.csv')
df_funds = pd.read_csv(data_folder + 'funds.csv')
df_investors = pd.read_csv(data_folder + 'investors.csv')
df_investments = pd.read_csv(data_folder + 'investments.csv')

In [13]:
df_funding_rounds = df_funding_rounds[['company_uuid', 'funding_round_uuid']]
df_funding_rounds.head()

,company_uuid,funding_round_uuid
0,4f503d57-bf03-4010-6fcc-036fab95b39c,d17ab961-d739-410e-e904-4cb78057cbe1
1,bb971d77-d7b8-a3c3-6925-c418077e5d3c,828b3a89-451e-cd0a-6d57-e31dd6a46124
2,0d4c4d64-7545-1a9b-b176-8cc2436abc50,5d2d7a43-d310-d880-6da1-49b9189ce36a
3,60a1f884-4b10-c925-4b43-04bd1aa3d98f,46d479cb-4867-33ed-2cb7-df679f9ec50d
4,289bddd6-ffc3-7fc5-7266-ba067d980288,350c268e-1878-15de-f221-9fd45548d4f7


In [14]:
df_investments = df_investments[['funding_round_uuid', 'investor_uuid']]
df_investments.head()

,funding_round_uuid,investor_uuid
0,0001cbd1-f7e2-4a56-607c-c57f46cc7dcb,3213cd7d-06e0-3ecb-3359-29aa8deddfa6
1,0001cbd1-f7e2-4a56-607c-c57f46cc7dcb,76c506a0-a56e-2f7b-9c15-2c4740ca51e1
2,0001cbd1-f7e2-4a56-607c-c57f46cc7dcb,960ce830-967e-79c4-e98a-b402d7db9748
3,00027faf-5c46-d1a5-0aa6-7649b3218166,e7e5a267-ce84-b580-ccbf-40a17099d2aa
4,0003c42b-498e-cd53-aec6-53461d667c79,39041e62-6b24-ae8d-1347-4cea947e832c


In [15]:
df = df_funding_rounds.merge(df_investments, on = 'funding_round_uuid')

In [16]:
groups = df.groupby('funding_round_uuid')['investor_uuid'].apply(list)

In [17]:
g = nx.Graph()
# groups = df_investments.groupby('funding_round_uuid')['investor_uuid'].apply(list)

In [20]:
# 같은 회사에 투자를 한 경우, 두 투자 회사 사이에 링크를 준다
g = nx.Graph()
for index, investors in enumerate(groups.values):
    for investor1 in investors:
        for investor2 in investors:
            if type(investor1) != str or type(investor2) != str:
                continue
            if investor1 != investor2:
                weight = 1
                if investor1 in g:
                    if investor2 in g[investor1]:
                        weight = g[investor1][investor2]['weight'] + 1
                g.add_edge(investor1, investor2, weight = weight)
with open(util_folder + 'investor_weight_network.pickle', 'w') as f:
    pickle.dump(g, f)

In [ ]:
g = pickle.load(open(util_folder + 'investor_weight_network.pickle'))
# for n1,n2,attr in g.edges(data=True):
#     print n1,n2,attr

# reverse weights
for edge in g.edges():
    n0 = edge[0]
    n1 = edge[1]
    weight = 1 / float(g[n0][n1]['weight'])
    g.add_edge(n0, n1, weight = weight)

print 'calculating centrality'
central = nx.betweenness_centrality(g, weight = 'weight')
central = sorted(central.items(), key=operator.itemgetter(1))
central.reverse()
f = open(util_folder + 'investor_centrality_between_one_over_k.pickle', 'w')
pickle.dump(central, f)
f.close()

calculating centrality


In [ ]:
central = pickle.load(open(util_folder + 'investor_centrality_between_one_over_k.pickle'))
top20 = [uuid for uuid, value in central[:10]]
for i, uuid in enumerate(top20):
    print df_investors[df_investors['uuid'] == (uuid)]['investor_name'], central[i][1]

In [ ]:
g = pickle.load(open(util_folder + 'investor_weight_network.pickle'))
# for n1,n2,attr in g.edges(data=True):
#     print n1,n2,attr

# reverse weights
for edge in g.edges():
    n0 = edge[0]
    n1 = edge[1]
    weight = 1 / np.sqrt(float(g[n0][n1]['weight']))
    g.add_edge(n0, n1, weight = weight)

print 'calculating centrality'
central = nx.betweenness_centrality(g, weight = 'weight')
central = sorted(central.items(), key=operator.itemgetter(1))
central.reverse()
f = open(util_folder + 'investor_centrality_between_one_over_sqrt_k.pickle', 'w')
pickle.dump(central, f)
f.close()

In [ ]:
central = pickle.load(open(util_folder + 'investor_centrality_between_one_over_sqrt_k.pickle'))
top20 = [uuid for uuid, value in central[:10]]
for i, uuid in enumerate(top20):
    print df_investors[df_investors['uuid'] == (uuid)]['investor_name'], central[i][1]

In [18]:
central = pickle.load(open(util_folder + 'investor_centrality_degree.pickle'))
top20 = [uuid for uuid, value in central[:10]]

In [19]:
central = pickle.load(open(util_folder + 'investor_centrality_degree.pickle'))
top20 = [uuid for uuid, value in central[:10]]for i, uuid in enumerate(top20):
    print df_investors[df_investors['uuid'] == (uuid)]['investor_name'], central[i][1]

0    500 Startups
Name: investor_name, dtype: object 0.0645051194539
3    New Enterprise Associates
Name: investor_name, dtype: object 0.0460750853242
12    SV Angel
Name: investor_name, dtype: object 0.0447886584405
4    Intel Capital
Name: investor_name, dtype: object 0.0431346810186
2    Y Combinator
Name: investor_name, dtype: object 0.0369650826989
5    Accel Partners
Name: investor_name, dtype: object 0.035914938304
7    Kleiner Perkins Caufield & Byers
Name: investor_name, dtype: object 0.034287214492
1    Sequoia Capital
Name: investor_name, dtype: object 0.0339459175637
32    GV
Name: investor_name, dtype: object 0.0312943029667
17    First Round
Name: investor_name, dtype: object 0.0301916513521
